In [1]:
# Import necessary libraries
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from textpreprocessor import TextPreprocessor

# Initialize the Text Pre Processor class
processor = TextPreprocessor()

# Load data
df_train, df_test = processor.load_data()

# Preprocess data
df_train = processor.preprocess(df_train)
df_test = processor.preprocess(df_test)

# Split data
X_train, y_train= processor.split_data(df_train)
X_test, y_test = processor.split_data(df_test)

X_train_tfidf, X_test_tfidf = processor.vectorize_text(X_train, X_test)

2024-09-19 22:29:15.766663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 22:29:15.775590: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 22:29:15.778214: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 22:29:15.785450: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-19 22:29:16.180442: W tensorflow/compiler/tf2

(3600000, 3)
(10000, 3)
(400000, 3)
(10000, 3)


In [2]:
# Convert to DMatrix (XGBoost's data format)
dtrain = xgb.DMatrix(X_train_tfidf, label=y_train)
dtest = xgb.DMatrix(X_test_tfidf, label=y_test)

In [3]:
# Define model parameters for XGBoost
params = {
    'objective': 'binary:logistic',  # Binary classification
    'max_depth': 6,
    'eta': 0.3,
    'verbosity': 1,
    'eval_metric': 'logloss',
    'tree_method': 'gpu_hist'  # Use GPU for training
}

In [4]:
# Train the model using GPU
model = xgb.train(params, dtrain, num_boost_round=100)

/home/woong/myenv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:29:25] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [5]:
# Make predictions
y_pred_prob = model.predict(dtest)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

/home/woong/myenv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:29:27] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [6]:
# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 80.66%
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.80      0.80      4972
           1       0.80      0.82      0.81      5028

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000

